In [26]:
import sqlite3
from IPython.display import Image

conn = sqlite3.connect('../database/database.db')

cursor = conn.cursor()


def query(input):
    
    cursor.execute(input)
    colnames = cursor.description
    for name in colnames:
        print(name[0])


    records = cursor.fetchall()

    for record in records:
        print(record)


In [28]:
query('''
      select *
      from OfficerDim
      ''')

Key
Id
Name
Rank
(19, '1', 'Joe', 'Sergeant')
(20, '2', 'Mike', 'Patrolman')
(21, '3', 'Bob', 'Patrolman')


In [30]:
query('''
      select * from vehicledim''')

Key
LPN
Make
Model
Year
OwnerDLN
OwnerName
OwnerGender
OwnerBirthYear
(1, 'IL2222', 'Honda', 'Civic', 2020, '111', 'Bill', 'Male', 1998)
(2, 'IL2222', 'Honda', 'Civic', 2020, '222', 'Suzy', 'Female', 1987)
(3, 'IL1111', 'Honda', 'CRV', 2017, '111', 'Bill', 'Male', 1998)


In [42]:
query('''
      select 
      
      case when id='a' then 'driving'
        else 'driving' end as Category,
      violation,
      fee 
      
      from drivingtickettype
    
      union
      
      select 
      
      case when id='a' then 'parking'
        else 'parking' end as Category,
      violation,
      fee
       
      from parkingtickettype

      ''')

Category
Violation
Fee
('driving', 'Red Light', 200)
('driving', 'Seat Belt', 100)
('parking', 'Hydrant', 150)
('parking', 'Meter Expired', 100)


In [44]:
query('select * from TicketTypeDim')

Key
Category
Violation
Fee
(1, 'driving', 'Red Light', 200)
(2, 'driving', 'Seat Belt', 100)
(3, 'parking', 'Hydrant', 150)
(4, 'parking', 'Meter Expired', 100)


In [71]:
query('''
      select
      -------------------------------------- case for days of week
      case cast (strftime('%w', date) as integer)
      when 0 then 'Sunday'
      when 1 then 'Monday'
      when 2 then 'Tuesday'
      when 3 then 'Wednesday'
      when 4 then 'Thursday'
      when 5 then 'Friday'
      else 'Saturday' end as DayOfWeek,
      strftime('%d', date) as DayOfMonth,
      strftime('%m', date) as Month,
      ------------------------------------- case for quarters
      case cast (strftime('m',date) as integer)
      when 1 | 2 | 3 then 'Q1'
      when 4 | 5 | 6 then 'Q2'
      when 7 | 8 | 9 then 'Q3'
      when 10 | 11 | 12 then 'Q4'
      else 'q4' end as Quarter,
      --------------------------------------
      
      strftime('%Y', date) as Year
      
      from DrivingTicket

      union all
      
      select
      -------------------------------------- case for days of week
      case cast (strftime('%w', date) as integer)
      when 0 then 'Sunday'
      when 1 then 'Monday'
      when 2 then 'Tuesday'
      when 3 then 'Wednesday'
      when 4 then 'Thursday'
      when 5 then 'Friday'
      else 'Saturday' end as DayOfWeek,
      strftime('%d', date) as DayOfMonth,
      strftime('%m', date) as Month,
      ------------------------------------- case for quarters
      case cast (strftime('m',date) as integer)
      when 1 | 2 | 3 then 'Q1'
      when 4 | 5 | 6 then 'Q2'
      when 7 | 8 | 9 then 'Q3'
      when 10 | 11 | 12 then 'Q4'
      else 'q4' end as Quarter,
      --------------------------------------
      
      strftime('%Y', date) as Year
      
      from ParkingTicket
      ''')

DayOfWeek
DayOfMonth
Month
Quarter
Year
('Wednesday', '01', '01', 'q4', '2020')
('Thursday', '02', '01', 'q4', '2020')
('Thursday', '02', '01', 'q4', '2020')
('Wednesday', '01', '01', 'q4', '2020')
('Wednesday', '01', '01', 'q4', '2020')
('Thursday', '02', '01', 'q4', '2020')


In [97]:
query('''
      select distinct date 
      from drivingticket
      
      union
      
      select distinct date
      from parkingticket
      ''')

Date
('2020-01-01',)
('2020-01-02',)


In [84]:
cursor.execute('delete from CalendarDim')
conn.commit()

In [101]:
query('select * from calendardim')

Key
Fulldate
DayOfWeek
DayOfMonth
Month
Quarter
Year
(33, '2020-01-01', 'Wednesday', 1, 1, 'q4', 2020)
(34, '2020-01-01', 'Thursday', 2, 1, 'q4', 2020)
(35, '2020-01-02', 'Wednesday', 1, 1, 'q4', 2020)
(36, '2020-01-02', 'Thursday', 2, 1, 'q4', 2020)


In [98]:
query('''
      with fulldate as (
            select distinct date 
            from drivingticket

            union

            select distinct date
            from parkingticket
      )
      
      select
      -------------------------------------- case for days of week
      case cast (strftime('%w', date) as integer)
      when 0 then 'Sunday'
      when 1 then 'Monday'
      when 2 then 'Tuesday'
      when 3 then 'Wednesday'
      when 4 then 'Thursday'
      when 5 then 'Friday'
      else 'Saturday' end as DayOfWeek,
      strftime('%d', date) as DayOfMonth,
      strftime('%m', date) as Month,
      ------------------------------------- case for quarters
      case cast (strftime('m',date) as integer)
      when 1 | 2 | 3 then 'Q1'
      when 4 | 5 | 6 then 'Q2'
      when 7 | 8 | 9 then 'Q3'
      when 10 | 11 | 12 then 'Q4'
      else 'q4' end as Quarter,
      --------------------------------------
      
      strftime('%Y', date) as Year
      
      from fulldate
      
      ''')

DayOfWeek
DayOfMonth
Month
Quarter
Year
('Wednesday', '01', '01', 'q4', '2020')
('Thursday', '02', '01', 'q4', '2020')


In [100]:
query('''
      with fulldate as (
            select distinct date 
            from drivingticket

            union

            select distinct date
            from parkingticket
      ),
      test as (
      select
      -------------------------------------- case for days of week
      case cast (strftime('%w', date) as integer)
      when 0 then 'Sunday'
      when 1 then 'Monday'
      when 2 then 'Tuesday'
      when 3 then 'Wednesday'
      when 4 then 'Thursday'
      when 5 then 'Friday'
      else 'Saturday' end as DayOfWeek,
      strftime('%d', date) as DayOfMonth,
      strftime('%m', date) as Month,
      ------------------------------------- case for quarters
      case cast (strftime('m',date) as integer)
      when 1 | 2 | 3 then 'Q1'
      when 4 | 5 | 6 then 'Q2'
      when 7 | 8 | 9 then 'Q3'
      when 10 | 11 | 12 then 'Q4'
      else 'q4' end as Quarter,
      --------------------------------------
      
      strftime('%Y', date) as Year
      
      from fulldate)
      select * from test
      
      ''')

DayOfWeek
DayOfMonth
Month
Quarter
Year
('Wednesday', '01', '01', 'q4', '2020')
('Thursday', '02', '01', 'q4', '2020')
